In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder  

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-may-2021/train.csv", index_col = 'id')
x_test_data = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")
x_test_new = x_test_data.iloc[:,1:51]

X = train.drop('target', axis = 1).astype('float32')
lencoder = LabelEncoder()
y = pd.DataFrame(lencoder.fit_transform(train['target']).astype('int32'))

No Missing Values

There seems to be no correlation across all the features

In [ ]:
from catboost import CatBoostClassifier,Pool

model = CatBoostClassifier(random_seed= 24,task_type="GPU",classes_count=4,loss_function='MultiClass',eval_metric='MultiClass',leaf_estimation_method='Newton')

grid = {'learning_rate': [0.002, 0.008, 0.01, 0.05, 0.1, 0.2, 0.5, 0.9],
        'depth': [4,5,6,8,10],
        'l2_leaf_reg': [0.5, 1, 7, 9, 12,14.6, 50.5],
        'bagging_temperature':[0,1],
        'border_count': [30,64,128, 254],
        'n_estimators': [200,400, 700, 900, 1200, 1500, 1800,2300]}

from sklearn.model_selection import RandomizedSearchCV


randm = RandomizedSearchCV(estimator=model, param_distributions = grid,cv = 5, n_iter = 50, n_jobs=-1)

#### Uncomment below to get the random search parameters
####################################
#randm.fit(X, y[0])
#param = randm.best_params_
#param

In [ ]:
param = {'n_estimators': 700,
 'learning_rate': 0.01,
 'l2_leaf_reg': 50.5,
 'depth': 12,
 'border_count': 50,
 'bagging_temperature': 1}

In [ ]:
clf = CatBoostClassifier(random_seed= 24,task_type="GPU",classes_count=4,loss_function='MultiClass',eval_metric='MultiClass',leaf_estimation_method='Newton',leaf_estimation_iterations = 10)
clf.set_params(**param)

%time
clf.fit(X,y)


In [ ]:
clf.score(X,y)

In [ ]:
y_pred_proba = clf.predict_proba(x_test_new).astype('float32')
output_df = pd.DataFrame(y_pred_proba, columns=['class_1', 'class_2', 'class_3', 'class_4'])
output_df = pd.concat([x_test_data.id,output_df],axis = 1)
output_df.head()

In [ ]:
output_df.to_csv('OutputXGB.csv', index=False)